In [1]:
class leastSquares:
     """
    This class implements the least squares approximation of a function f(x) over [-1, 1] using the 
    Legendre polynomials. It computes the coefficients of the approximation and the corresponding errors. 
    The class can also print the values of the beta, gamma and mu coefficients, as well as the coefficients, 
    L2 error, and max error for each degree of the approximation.
    
    Parameters:
    -----------
    N : int
        The degree of the approximation (i.e. the number of Legendre polynomials to be used).
    f : callable
        The function to be approximated. It should take two arguments, n and t, where n is the degree of the 
        polynomial and t is the evaluation point.
    
    Methods:
    --------
    t_i(i):
        Computes the ith Gauss-Legendre quadrature point over the interval [-1, 1].
        
    b_k(k):
        Computes the beta coefficients for the k-th Legendre polynomial.
        
    y_k(k):
        Computes the gamma coefficients for the k-th Legendre polynomial.
        
    pi_k(k, t):
        Computes the k-th Legendre polynomial evaluated at t using the recurrence relation.
        
    inner_prod(u, v, k):
        Computes the inner product of the u and v functions for the k-th Legendre polynomial.
        
    u_k(k):
        Computes the maximum value of the k-th Legendre polynomial over [-1, 1].
        
    p_n(n, t):
        Computes the least squares approximation of f(n, t) using n Legendre polynomials.
        
    c_k(k):
        Computes the k-th coefficient of the least squares approximation.
        
    en_2(n):
        Computes the L2 error of the least squares approximation using n Legendre polynomials.
        
    e_infinity(n):
        Computes the max error of the least squares approximation using n Legendre polynomials.
        
    printB():
        Prints the values of the beta, gamma and mu coefficients for each Legendre polynomial.
        
    printC():
        Prints the coefficients, L2 error, and max error for each degree of the least squares approximation.
    """
    
    def __init__(self, N, f):
        self.N = N
        self.f = f
        
    def t_i(self, i):
        return (-1+2*i/self.N)
    
    def b_k(self, k):
        if (k==0):
            return 2
        else:
            return ((1+(1/self.N))**2*(1-(k/(self.N+1))**2)*(4-(1/(k**2)))**(-1))
        
    def y_k(self, k):
        x=1
        for i in range(k+1):
            x=x*self.b_k( i)
        return x
    
    def pi_k(self, k, t):
    
        if (k==-1):
            return 0
        elif (k==0):
            return 1
        else:
            return (self.t_i(t)*self.pi_k(k-1,t)-self.b_k(k-1)*self.pi_k(k-2,t))
        
    def inner_prod(self, u, v, k):
        a=[]
        for i in range (self.N+1):
            b=(2/(self.N+1))*u(k,i)*v(k,i)
            a.append(b)
        return(sum(a))
    
    def u_k(self, k):
        a=[]
        for i in range (self.N+1):
            a.append(abs(self.pi_k(k,i)))
        return(max(a))

    def p_n(self, n, t):
        a=[]
        for k in range (n+1):
            a.append(self.c_k(k)*self.pi_k(k,t))
        return(sum(a))

    def c_k(self, k):
        return (self.inner_prod(self.pi_k, self.f, k)/self.y_k(k))


    def en_2(self, n):
        a=[]
        for i in range (self.N+1):
            b=((2/(self.N+1))*(self.p_n(n,i)-self.f(n,i))*(self.p_n(n,i)-self.f(n,i)))
            a.append(b)
        return(sum(a)**(1/2))

    def e_infinity(self, n):

        a=[]
        for t in range (self.N+1):
            a.append(self.p_n(n, t)-self.f(n, t))
        return(max(a))
    
    def printB(self):
        A=[self.b_k(i) for i in range(self.N+1)]
        B=[self.y_k(i) for i in range(self.N+1)]
        C=[self.u_k(i) for i in range(1, self.N+2)]
        df = pd.DataFrame(list(zip(A, B, C)),
               columns =['Beta K', 'Gamma K', 'Mu K'])
        print(df)
        
    def printC(self):
        A=[self.c_k(i) for i in range(self.N+1)]
        B=[self.en_2(i) for i in range(self.N+1)]
        C=[self.e_infinity(i) for i in range(self.N+1)]
        df = pd.DataFrame(list(zip(A, B, C)),
               columns =['Coefficients', 'L2 Error', 'Max Error'])
        print(df)

In [2]:
import numpy as np
N=10

def t_i(i):
    return (-1+2*i/N)

def f1(k, t):
    return np.exp(-t_i(t))

def f2(k, t):
    return np.log(2+t_i(t))
    
def f3(k, t):
    return (1+t_i(t))**(1/2)

def f4(k, t):
    return abs(t_i(t))

leastSquares(10, f1).printB()

for i in [f1, f2, f3, f4]:
    leastSquares(10, i).printC()

      Beta K       Gamma K          Mu K
0   2.000000  2.000000e+00  1.000000e+00
1   0.400000  8.000000e-01  6.000000e-01
2   0.312000  2.496000e-01  2.880000e-01
3   0.288000  7.188480e-02  1.152000e-01
4   0.266667  1.916928e-02  7.680000e-02
5   0.242424  4.647098e-03  3.351273e-02
6   0.213986  9.944140e-04  1.488738e-02
7   0.180923  1.799124e-04  5.269232e-03
8   0.143059  2.573806e-05  1.834253e-03
9   0.100310  2.581775e-06  5.068331e-04
10  0.052632  1.358829e-07  2.697038e-17
    Coefficients      L2 Error     Max Error
0   1.212204e+00  1.042186e+00  8.443242e-01
1  -1.123748e+00  2.755052e-01  2.155505e-01
2   5.430256e-01  4.797789e-02  4.639123e-02
3  -1.774968e-01  6.094239e-03  5.763659e-03
4   4.380735e-02  5.935399e-04  7.170521e-04
5  -8.681309e-03  4.539208e-05  5.032757e-05
6   1.436823e-03  2.741009e-06  2.907943e-06
7  -2.041262e-04  1.289394e-07  1.298950e-07
8   2.539983e-05  4.518453e-09  5.224383e-09
9  -2.811358e-06  1.032893e-10  1.420133e-10
10  2.800873e